In [125]:
# Import the data
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

df = pd.read_csv('data/valentine_gift_large_dataset.csv')
df.head()


,Recipient_Gender,Recipient_Age,Relationship_Status,Budget,Occasion,Preference,Relationship_Length,Personal_Interest,Special_Note,Gift_Popularity,Gift_Purpose,Past_Gift_Reaction,Delivery_Method,Gift_Wrapping_Preference,Best_Gift
0,Female,29,Single,High ($100-$500),Anniversary Valentine,Sentimental,<6 months,Fitness,Has allergies,Very Popular,Celebration,Disliked,Online Delivery,Fancy Wrapping,Perfume
1,Other,34,Married,Medium ($20-$100),First Valentine,Sentimental,3+ years,Music,Handmade gifts appreciated,Moderately Popular,Celebration,Highly Appreciated,Planned Surprise,No Wrapping,Fashion Accessories
2,Other,56,Single,Low (<$20),First Valentine,Practical,3+ years,Fashion,Tech-savvy,Moderately Popular,Expression of Love,Never Given Gift Before,Online Delivery,Eco-Friendly,Flowers
3,Other,50,Complicated,Luxury (>$500),First Valentine,Fun,6 months - 1 year,Cooking,Prefers practical gifts,Very Popular,Expression of Love,Disliked,Online Delivery,Fancy Wrapping,Fashion Accessories
4,Male,53,Married,Medium ($20-$100),Long-Distance Valentine,Surprise,<6 months,Music,Has allergies,Moderately Popular,Apology,Highly Appreciated,Planned Surprise,Eco-Friendly,Subscription Service


In [126]:
# merge two samilar value to one vlaue
def mergeData(df, column_name, value_one, value_two):
    df[column_name] = df[column_name].replace(value_one, value_two)

In [127]:
# encode column's value
def hot_encode(df):
       categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
       encoder = OneHotEncoder(sparse_output=False)
       one_hot_encoded = encoder.fit_transform(df[categorical_columns])

       one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
       df_encoded = pd.concat([df, one_hot_df], axis=1)
       df_encoded = df_encoded.drop(categorical_columns, axis=1)

       return df_encoded

In [128]:
#  ordinal encode y values
# def label_encode(df):
#     encoder = LabelEncoder(categories=[df['Best_Gift'].unique()])
#     df['Best_Gift_Encoded'] = encoder.fit_transform(df[['Best_Gift']])
#     print(df[['Best_Gift', 'Best_Gift_Encoded']].head())
#     return df['Best_Gift_Encoded']


from sklearn import preprocessing 
  
def label_encode(df):
# label_encoder object knows  
# how to understand word labels. 
    label_encoder = preprocessing.LabelEncoder() 
    
    # Encode labels in column 'species'. 
    df['Best_Gift_Encoded']= label_encoder.fit_transform(df['Best_Gift']) 
    
    df['Best_Gift_Encoded'].unique() 
    print(df[['Best_Gift', 'Best_Gift_Encoded']].head())

    return df['Best_Gift_Encoded']
 
          


In [ ]:
def clean_data(df):
    
    # drop useless columns
    df = df.drop(columns = ['Special_Note', 'Past_Gift_Reaction', 'Delivery_Method', 'Gift_Wrapping_Preference'])

    # combine the samiliar values for few columns
    mergeData(df, "Recipient_Gender", 'Non-Binary', 'Other')
    mergeData(df, "Preference", "Sentimental", "Romantic")
    mergeData(df, "Personal_Interest", "Sports", "Fitness")
    mergeData(df, "Personal_Interest", "Gaming", "Technology")
    mergeData(df, "Best_Gift", "Handmade Gift", "Personalized Gift")
    mergeData(df, "Best_Gift", "Chocolates", "Chocolates & Flowers")
    mergeData(df, "Best_Gift", "Flowers", "Chocolates & Flowers")
    mergeData(df, "Best_Gift", "Jewelry", "Fashion Accessories")


    # split X and y
    # use hot encoding to encode some columns in x 
    X = df.drop(columns=['Best_Gift'])
    X = hot_encode (X)

    # use OrdinalEncoder to encode "Best_Gift" column and get y
    y = label_encode(df)

    
    return X, y









  




In [ ]:

# Models to use in our pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler



from sklearn.tree import DecisionTreeRegressor

# Preprocessing dependencies
from sklearn.model_selection import train_test_split


# Metrics
from sklearn.metrics import mean_squared_error, r2_score

def r2_adj(X, y, model):
    """
    Calculates adjusted r-squared values

    Args:
    X: Independent variables, the data to fit
    y: dependent variable, the target data to try to predict
    model: The estimator or object to use to train the data

    Returns: adjusted r sqaured value accountign for number of predictors
    """
    r2 = model.score(X, y)
    n = X.shape[0]
    p = y.ndim


def model_generator(X, y):
    y = y.values.reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

    models = {
        "RF": RandomForestClassifier(),
        'DT': DecisionTreeClassifier(),
        'SVC': SVC(),
        'AB': AdaBoostClassifier()
    }


    for  name, model in models.items():
        pipeline = Pipeline([
            ("Scale", StandardScaler(with_mean=False)),
            (name, model)
        ])

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        print("name:", name, "score:", pipeline.score(X_test, y_test))

    


In [131]:
X, y = clean_data(df)

model_generator(X, y)

              Best_Gift  Best_Gift_Encoded
0               Perfume                  3
1   Fashion Accessories                  2
2  Chocolates & Flowers                  0
3   Fashion Accessories                  2
4  Subscription Service                  5


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


name: RF score: 0.192
name: DT score: 0.146


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


name: SVC score: 0.19


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


name: AB score: 0.178
